# Imitation Learning over Heterogeneous Agents with Restraining Bolts
## Project development

Source: (De Giacomo, 2019)[http://www.diag.uniroma1.it/degiacom/papers/2020/icaps2020dfip.pdf]   

This notebook intends to run one the examples from the example provided code of the implementation of the research work paper.


# Breakout

The reference of the following code is the file: `breakout/__main__.py`, with slight modifications to be executed in this notebook. The goal is to show the flow and values of variables across the execution.

In [17]:
"""This is the main entry-point for the experiments with the Breakout environment."""
import logging
import os
from argparse import ArgumentParser

import yaml

from breakout.learner import run_learner
from breakout.expert import run_expert
from rl_algorithm.utils import Map, learn_dfa



In [2]:
## Show video method

from IPython import display as ipythondisplay
from IPython.display import HTML
import io
import base64
import glob

def show_video(list_paths, take_first=True):
    '''
    This method allows to show a list of videos next to each other, for comparison purposes.
    
    
    '''
    
    encoded_videos = []
    for i, video_path in enumerate(list_paths):
                
        mp4list = glob.glob(video_path + 'videos/*.mp4')
        ind = 0
        if len(mp4list) > 0:
            if not take_first:
                 ind = -1

            mp4 = mp4list[ind]
            print(i+1,': ', mp4)
            video = io.open(mp4, 'r+b').read()
            encoded = base64.b64encode(video)
            encoded_videos.append('''<video alt="test" autoplay 
                        loop controls style="height: 400px;">
                        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                     </video> '''.format(encoded.decode('ascii')))
        else:
            print("Could not find video of path: ", video_path)
    
    if len(encoded_videos) > 0:
        ipythondisplay.display(HTML(data='''<div>''' \
                                        + ''.join(encoded_videos) \
                                        + '''</div>'''))

In [3]:
# Preview. This wouldn't work if it's the first time the trainings are going to be executed, since the files wouln'dt exist
show_video(['experiments/breakout-output/expert/',
            'experiments/breakout-output/learner/'], False)

1 :  experiments/breakout-output/expert/videos\openaigym.video.1.4670.video000001.mp4
2 :  experiments/breakout-output/learner/videos\openaigym.video.2.4670.video000001.mp4


In [3]:
logging.getLogger("temprl").setLevel(level=logging.DEBUG)
logging.getLogger("matplotlib").setLevel(level=logging.INFO)
logging.getLogger("rl_algorithm").setLevel(level=logging.INFO)

## Argument parsing
Manually add some default values to run the example.
* The suggested line in CLI is:
`python3 -m breakout --rows 3 --cols 3 --output-dir experiments/breakout-output --overwrite`

* Change `Overwrite` to `True` in arguments, as the experiments does not exist yet. This option **deletes any training** previously performed.


* The specific arguments of the expert and the learner are in the `.yaml` files

In [27]:
parser = ArgumentParser()
parser.add_argument("--cols", type=int, default=3, help="Number of columns.")
parser.add_argument("--rows", type=int, default=3, help="Number of rows.")
parser.add_argument("--brick-reward", type=int, default=5, help="The reward for breaking a brick.")
parser.add_argument("--step-reward", type=float, default=-0.01, help="The reward for breaking a brick.")
parser.add_argument("--goal-reward", type=int, default=1000, help="The reward for satisfying the temporal goal.")
parser.add_argument("--overwrite", action="store_true", default=True, help="Overwrite the content of the output directory.")
parser.add_argument("--seed", type=int, default=42, help="Random seed.")
parser.add_argument("--output-dir", type=str, default="experiments/breakout-output", help="Output directory for the experiment results.")
parser.add_argument("--expert-config", type=str, default="breakout/expert_config.yaml", help="RL configuration for the expert.")
parser.add_argument("--learner-config", type=str, default="breakout/learner_config.yaml", help="RL configuration for the learner.")
arguments = parser.parse_args(args=[])
arguments

Namespace(brick_reward=5, cols=3, expert_config='breakout/expert_config.yaml', goal_reward=1000, learner_config='breakout/learner_config.yaml', output_dir='experiments/breakout-output', overwrite=True, rows=3, seed=42, step_reward=-0.01)

# Main

In [28]:
expert_config = Map(yaml.safe_load(open(arguments.expert_config)))
learner_config = Map(yaml.safe_load(open(arguments.learner_config)))

### Run the expert

Some facts about this training:
* It implements either SARSA or Q-Learning as training algorithm.
* The specific arguments of the expert and the learner are in the .yaml files.
* The actual algorithms, SARSA and Q-Learning, are referred to in the code as 'brain', and are developed in the `brains.py` file. And this are used by the `Agent`.
* The experts uses the `true_automaton` (diagram shown below, in Automaton) to learn the goal.

`expert.py (just this method):`

``` python 

def run_expert(arguments, configuration):
    agent_dir = Path(arguments.output_dir) / "expert"
    if arguments.overwrite:
        shutil.rmtree(arguments.output_dir, ignore_errors=True)
    agent_dir.mkdir(parents=True, exist_ok=False)

    config = BreakoutConfiguration(brick_rows=arguments.rows, brick_cols=arguments.cols,
                                   brick_reward=arguments.brick_reward, step_reward=arguments.step_reward,
                                   ball_enabled=False, fire_enabled=True)
    env = make_env(config, arguments.output_dir, arguments.goal_reward)

    np.random.seed(arguments.seed)
    env.seed(arguments.seed)

    policy = AutomataPolicy((-2, ), nb_steps=configuration.nb_exploration_steps, value_max=1.0, value_min=configuration.min_eps)

    algorithm = Sarsa if configuration.algorithm == "sarsa" else QLearning
    agent = Agent(algorithm(None,
                        env.action_space,
                        gamma=configuration.gamma,
                        alpha=configuration.alpha,
                        lambda_=configuration.lambda_),
                  policy=policy,
                  test_policy=EpsGreedyQPolicy(eps=0.01))

    history = agent.fit(
        env,
        nb_steps=configuration.nb_steps,
        visualize=configuration.visualize_training,
        callbacks=[
            ModelCheckpoint(str(agent_dir / "checkpoints" / "agent-{}.pkl")),
            TrainEpisodeLogger()
        ]
    )
    history.save(agent_dir / "history.json")
    agent.save(Path(agent_dir, "checkpoints", "agent.pkl"))
    plot_history(history, agent_dir)

    agent = Agent.load(agent_dir / "checkpoints" / "agent.pkl")
    agent.test(Monitor(env, agent_dir / "videos"), nb_episodes=5, visualize=True)

    env.close()
```

In [29]:
expert_config

{'nb_steps': 75000,
 'nb_exploration_steps': 10000,
 'min_eps': 0.01,
 'visualize_training': False,
 'reward_shaping': True,
 'gamma': 0.99,
 'alpha': 0.1,
 'lambda_': 0.0,
 'algorithm': 'sarsa'}

In [30]:
print("Run the expert.")
run_expert(arguments, expert_config)

Run the expert.


[2020-05-11 08:44:52,416][graphviz.files][save][DEBUG]: write 1117 bytes to 'experiments/breakout-output/true_automaton'
[2020-05-11 08:44:52,427][graphviz.backend][run][DEBUG]: run ['dot', '-Tsvg', '-O', 'true_automaton']
[2020-05-11 08:44:52,593][temprl.automata][step][DEBUG]: transition idxs: 0, 2
[2020-05-11 08:44:52,602][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [-333.3333333333333, 0.0]


Formula: <(!c0 & !c1 & !c2)*;c0;(!c0 & !c1 & !c2)*;c1;(!c0 & !c1 & !c2)*;c2>tt
Original automaton at experiments/breakout-output/true_automaton.svg
Training for 75000 steps ...


[2020-05-11 08:44:54,172][temprl.automata][step][DEBUG]: transition idxs: 0, 2
[2020-05-11 08:44:54,182][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [-333.3333333333333, 0.0]


  2549/75000: episode: 1, duration:  1.483s, episode steps:  2549, steps per second:  1718, episode reward:   -313.823, mean reward:  -0.123 [  -328.343,      4.990], mean action:  1.475 [ 0.000,  3.000], mean observation:  1.612 [ 0.000,  9.000], q_value:      0.017, mean-eps:      0.937


...

[2020-05-11 08:46:06,110][temprl.automata][step][DEBUG]: transition idxs: 0, 1
[2020-05-11 08:46:06,115][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.3333333333333, 0.0]
[2020-05-11 08:46:06,403][temprl.automata][step][DEBUG]: transition idxs: 1, 3
[2020-05-11 08:46:06,408][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.3333333333333, 0.0]
[2020-05-11 08:46:06,696][temprl.automata][step][DEBUG]: transition idxs: 3, 4
[2020-05-11 08:46:06,700][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.33333333333337, 0.0]


Episode 4: reward: 1041.440, steps: 356


[2020-05-11 08:46:07,027][temprl.automata][step][DEBUG]: transition idxs: 0, 1
[2020-05-11 08:46:07,031][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.3333333333333, 0.0]
[2020-05-11 08:46:07,326][temprl.automata][step][DEBUG]: transition idxs: 1, 3
[2020-05-11 08:46:07,332][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.3333333333333, 0.0]
[2020-05-11 08:46:07,626][temprl.automata][step][DEBUG]: transition idxs: 3, 4
[2020-05-11 08:46:07,630][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.33333333333337, 0.0]


Episode 5: reward: 1041.430, steps: 357


<Figure size 432x288 with 0 Axes>

### Automaton

* DFA = Deterministic Finite-state Automaton.
* The expert learns form a true automaton, shown below.

**True Automaton**

![](experiments/breakout-output/true_automaton.svg)

**Learned Automaton**

<img src="experiments/breakout-output/learned_automaton.svg" alt="drawing" width="300"/>

In [31]:
print("Learn the automaton from traces.")
dfa = learn_dfa(arguments)

[2020-05-11 08:47:34,957][inferrer][__init__][INFO]: Created Active Learner [LSTAR] instance with Passive oracle
[2020-05-11 08:47:34,959][inferrer][learn][INFO]: Start learning.
[2020-05-11 08:47:34,959][inferrer][_initialise][INFO]: Initialising the table.
[2020-05-11 08:47:34,960][inferrer][_build_automaton][INFO]: Building DFA from the table.
[2020-05-11 08:47:34,961][inferrer][learn][INFO]: Submitting equivalence query.
[2020-05-11 08:47:34,962][inferrer][learn][INFO]: Oracle return 012 as counterexample.
[2020-05-11 08:47:34,966][inferrer][_useq][INFO]: Updating table by adding the following prefixes: , 01, 0, 012
[2020-05-11 08:47:34,967][inferrer][_consistent][INFO]: Making the table consistent by adding a column.
[2020-05-11 08:47:34,970][inferrer][_find_inconsistent][INFO]: Trying to find two inconsistent rows in the table.
[2020-05-11 08:47:34,971][inferrer][_find_inconsistent][INFO]: Found two inconsistent rows  and 01
[2020-05-11 08:47:34,971][inferrer][_consistent][INFO]:

Learn the automaton from traces.


In [41]:
dfa_dot_file = os.path.join(arguments.output_dir, "learned_automaton")
dfa.to_dot(dfa_dot_file)
print("Check the file {}.svg".format(dfa_dot_file))

[2020-05-11 09:01:50,090][graphviz.files][save][DEBUG]: write 464 bytes to 'experiments/breakout-output/learned_automaton'
[2020-05-11 09:01:50,097][graphviz.backend][run][DEBUG]: run ['dot', '-Tsvg', '-O', 'learned_automaton']


Check the file experiments/breakout-output/learned_automaton.svg


### Learner

Method from `learner.py` :
``` python

def run_learner(arguments, configuration, dfa: pythomata.dfa.DFA):
    agent_dir = Path(arguments.output_dir) / "learner"
    shutil.rmtree(agent_dir, ignore_errors=True)
    agent_dir.mkdir(parents=True, exist_ok=False)

    config = BreakoutConfiguration(brick_rows=arguments.rows, brick_cols=arguments.cols,
                                   brick_reward=arguments.brick_reward, step_reward=arguments.step_reward,
                                   fire_enabled=False, ball_enabled=True)
    env = make_env_from_dfa(config, dfa)

    np.random.seed(arguments.seed)
    env.seed(arguments.seed)

    policy = AutomataPolicy((-1, ), nb_steps=configuration.nb_exploration_steps, value_max=0.8, value_min=configuration.min_eps)

    algorithm = Sarsa if configuration.algorithm == "sarsa" else QLearning
    agent = Agent(algorithm(None,
                            env.action_space,
                            gamma=configuration.gamma,
                            alpha=configuration.alpha,
                            lambda_=configuration.lambda_),
                  policy=policy,
                  test_policy=EpsGreedyQPolicy(eps=0.001))

    history = agent.fit(
        env,
        nb_steps=configuration.nb_steps,
        visualize=configuration.visualize_training,
        callbacks=[
            ModelCheckpoint(str(agent_dir / "checkpoints" / "agent-{}.pkl")),
            TrainEpisodeLogger()
        ]
    )

    history.save(agent_dir / "history.json")
    agent.save(agent_dir / "checkpoints" / "agent.pkl")
    plot_history(history, agent_dir)

    agent = Agent.load(agent_dir / "checkpoints" / "agent.pkl")
    agent.test(Monitor(env, agent_dir / "videos"), nb_episodes=5, visualize=True)

    env.close()
```

In [97]:
learner_config

{'nb_steps': 75000,
 'nb_exploration_steps': 10000,
 'min_eps': 0.01,
 'visualize_training': False,
 'reward_shaping': True,
 'gamma': 0.99,
 'alpha': 0.1,
 'lambda_': 0.99,
 'algorithm': 'sarsa'}

In [42]:
print("Running the learner.")
run_learner(arguments, learner_config, dfa)

Running the learner.
Training for 75000 steps ...
   151/75000: episode: 1, duration:  0.245s, episode steps:   151, steps per second:   616, episode reward:      3.490, mean reward:   0.023 [    -0.010,      4.990], mean action:  0.848 [ 0.000,  2.000], mean observation:  7.614 [ 0.000,  47.000], q_value:     -0.000, mean-eps:      0.794
   440/75000: episode: 2, duration:  0.513s, episode steps:   289, steps per second:   564, episode reward:      7.110, mean reward:   0.025 [    -0.010,      4.990], mean action:  0.682 [ 0.000,  2.000], mean observation:  7.262 [ 0.000,  47.000], q_value:      0.001, mean-eps:      0.777
   591/75000: episode: 3, duration:  0.359s, episode steps:   151, steps per second:   421, episode reward:      3.490, mean reward:   0.023 [    -0.010,      4.990], mean action:  0.742 [ 0.000,  2.000], mean observation:  7.909 [ 0.000,  47.000], q_value:      0.003, mean-eps:      0.759
   742/75000: episode: 4, duration:  0.160s, episode steps:   151, steps per 

  8999/75000: episode: 29, duration:  0.380s, episode steps:   281, steps per second:   739, episode reward:      7.190, mean reward:   0.026 [    -0.010,      4.990], mean action:  0.658 [ 0.000,  2.000], mean observation:  7.587 [ 0.000,  47.000], q_value:      0.312, mean-eps:      0.098
  9280/75000: episode: 30, duration:  0.350s, episode steps:   281, steps per second:   803, episode reward:      7.190, mean reward:   0.026 [    -0.010,      4.990], mean action:  0.509 [ 0.000,  2.000], mean observation:  7.242 [ 0.000,  47.000], q_value:      0.287, mean-eps:      0.076
  9431/75000: episode: 31, duration:  0.124s, episode steps:   151, steps per second:  1214, episode reward:      3.490, mean reward:   0.023 [    -0.010,      4.990], mean action:  0.291 [ 0.000,  2.000], mean observation:  7.496 [ 0.000,  47.000], q_value:      0.126, mean-eps:      0.059
  9850/75000: episode: 32, duration:  0.679s, episode steps:   419, steps per second:   617, episode reward:     10.810, mea

[2020-05-11 09:02:42,771][temprl.automata][step][DEBUG]: transition idxs: 0, 1
[2020-05-11 09:02:42,781][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.3333333333333]


 10696/75000: episode: 34, duration:  1.144s, episode steps:   565, steps per second:   494, episode reward:    347.683, mean reward:   0.615 [    -0.010,    338.323], mean action:  0.411 [ 0.000,  2.000], mean observation:  7.112 [ 0.000,  47.000], q_value:      0.283, mean-eps:      0.064
 11255/75000: episode: 35, duration:  1.220s, episode steps:   559, steps per second:   458, episode reward:     14.410, mean reward:   0.026 [    -0.010,      4.990], mean action:  0.386 [ 0.000,  2.000], mean observation:  7.293 [ 0.000,  47.000], q_value:      1.023, mean-eps:      0.402


[2020-05-11 09:02:45,225][temprl.automata][step][DEBUG]: transition idxs: 0, 1
[2020-05-11 09:02:45,233][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.3333333333333]


...

[2020-05-11 09:06:18,634][temprl.automata][step][DEBUG]: transition idxs: 0, 1
[2020-05-11 09:06:18,638][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.3333333333333]
[2020-05-11 09:06:19,139][temprl.automata][step][DEBUG]: transition idxs: 1, 2
[2020-05-11 09:06:19,146][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.3333333333333]
[2020-05-11 09:06:19,699][temprl.automata][step][DEBUG]: transition idxs: 2, 3
[2020-05-11 09:06:19,701][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.33333333333337]


Episode 4: reward: 1035.890, steps: 911


[2020-05-11 09:06:20,916][temprl.automata][step][DEBUG]: transition idxs: 0, 1
[2020-05-11 09:06:20,919][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.3333333333333]
[2020-05-11 09:06:21,468][temprl.automata][step][DEBUG]: transition idxs: 1, 2
[2020-05-11 09:06:21,472][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.3333333333333]
[2020-05-11 09:06:21,994][temprl.automata][step][DEBUG]: transition idxs: 2, 3
[2020-05-11 09:06:21,999][temprl.wrapper][step][DEBUG]: Non-zero goal rewards: [333.33333333333337]


Episode 5: reward: 1035.890, steps: 911


<Figure size 432x288 with 0 Axes>

In [2]:
show_video(['experiments/breakout-output/expert/',
            'experiments/breakout-output/learner/'], False)

1 :  experiments/breakout-output/expert/videos\openaigym.video.1.4670.video000001.mp4
2 :  experiments/breakout-output/learner/videos\openaigym.video.2.4670.video000001.mp4


## Conclusions and facts

Some highlighted facts that can be drawn from these trainings are: 

* The expert and the learner, in this case, run the same algorithm (SARSA) and have the exact same parameters for the agent training.
* The environments change: The expert is trained with 'fire' mode, but the learner with 'ball' mode. This can be clearly seen in the videos.

![](experiments/breakout-output/Expert_Learner.png)